# Modellering av en Hohmann-bane i python

### Innledning (bakgrunn+forklaring av Hohmann-bane + presentere problem (Yang))
Allerede på 1960-tallet ble den første romsonden sendt i retning mars (Torgersen, 2018). Siden har forskere både klart å sende sonder i bane og landet rovere på planetes overflate (Aftenposten, 2021). Det neste på agendaen er å sende mennesker til den røde planeten. I dag investeres det enorme summer i dette prosjektet, både av offentlige aktører som NASA (USA) og CNSA (Kina), men også private aktører med Elon Musk og Jeff Besos i spissen (Bakken, 2024). Problemet er at en reise til mars vil være ekstremt dyr og utfordrende. Selve reisen tar over et halvt år. () Siden Mars bruker to jord-år rundt solen, vil det gå to år mellom hver gang planeten er i en gunstig posisjon for å sende et romskip. Det betyr at mennesker må tilbringe 2 år på Mars ved en eventuell reise før de kan dra tilbake til jorda. Planeten har verken vann eller oksygen. Alt man trenger for å overleve må være med fra jorda. Ferden til et romskip med såpass stor last vil kreve enorme mengder energi, og det vil være veldig dyrt.

Ifølge NASA vil en Hohmann-bane være den minst energikrevende og mest kostnadseffektive metoden for å sende et romfartøy til mars. Det er en metode som brukes for å overføre et romskip mellom to baner med forskjellig radius rundt et samme sentrallegeme. Ved å sende romfartøyet i en elliptisk overføringsbane (hyperbolic orbit) som tangerer både den opprinnelige banen og endebane, trengs bare to «impulsive motorforbrenninger». Det første for å overgå utslippningsfarten og dermed sende romskipet inn i Hohmann-banen. Den andre for å justere banen til romskipet slik at den matcher banen til målet. I dette prosjektet skal vi utforske fysikken bak en Hohmann-bane modellere en forenklet simulering i python. 


### <u>Antagelser</u> (Yang + Mohand)

Romskipet er allerede i bane rundt jorda

Banen til både jorda og mars er sirkulære med sola i sentrum. (begge er egentlig litt elliptiske)

Jorda og mars beveger seg med konstant hastighet.
Banen til jorda og mars er i samme plan.

Rommet er 2D

Kun tyngdekrafter spiller

Rommet er et vakuum

##### Setter opp programmet
Vi starter med å importere nødvendige python biblioteker: pygame og sys for å animere modellen, og math for matematiske operasjoner.
Videre må vi også oppgi en ønsket skjermoppløsning til programmet. 

In [97]:
import pygame, math, sys

# Skjermøppløsning til pygame (modulet som animerer)
SKJERM_BREDDE = 900
SKJERM_HØYDE = 700


Deretter definerer vi konstanter. 

In [98]:

astronomisk_enhet = 1.50 * 10**11

# Skalaen på animasjonen av programmet
SKALA = 200/astronomisk_enhet

#Masser
SOL_MASSE = 1.9891 * 10**30 
MARS_MASSE = 6.39 * 10**23 
JORD_MASSE = 5.9742 * 10**24 

AVSTANDJORD = astronomisk_enhet # Avstanden fra jorda til sol
AVSTANDMARS = astronomisk_enhet * 1.52 #Avstanden fra mars til sola

#Andre konstanter
GRAVITASJONS_KONSTANT = 6.67e-11 

TIDS_STEG = 60**2 # Programmet går på tidsteg per time - Kilde til usikkerhet

##### Klassen legeme
For å unngå at vi repeterer samme kode har vi valgt å definere himmelegemer som python objekter. Dette holder koden gjenbrukbar og hensiktsmessig. Dersom vi skal modellere bevegelsen til himmellegemer, er det noen faktorer vi først må ta hensyn til:


##### Kreftene som virker på legemet

Ved å bruke newtons gravitasjonslovbestemme kan vi bestemme kreftene som virker på et legeme:

 *Formel 1*:
 
$∑F_{j} = G =\frac{γM_{j}M_{s}}{{r_{1}}^2}$

Videre må vi dekomponere kreftene i henhold til x og y:


*Formler 2 og 3:*

$∑{F_{j}}_{x} = Gcos(θ) = \frac{γM_{j}M_{s}}{{r_{1}}^2}\times{cos(θ)}$

$∑{F_{j}}_{y} = Gsin(θ) = \frac{γM_{j}M_{s}}{{r_{1}}^2}\times{sin(θ)}$


Metoden *tyndgekraft()* i programmet benytter disse formlene for å beregne x- og y-kreftene som virker mellom legemer 

##### Fart og posisjon

Siden vi kjenner til kreftene som virker på et legeme, kan vi beregene akselrasjonen ved hjelp Newtons 2. lov:

F = ma
a = F/m

Hvis vi kjenner til akselrasjonen, kan vi beregne farten med formelen siden vi hele tiden beregner momentanakselrasjonen:

v_x = v_x0 + a_x * t
v_y = v_y0 + a_y * t

Og med det kan vi beregne posisjonen med

s_x = s_x0 + v_x*t
s_y = s_y0 + v_y*t

Metoden *oppdaterPosisjon()* i programmet benytter dette for å beregne posisjonen til legemet. 


##### Retning

<img src="VinkelVektor.png" alt="vinkelen mellom to vektorerer">


In [99]:
class Legeme():
    """
    En generell klasse for himmellegemer som inneholder metodene:
    \n tyngdekraft() for å beregne krefter som virker på legemet
    \n oppdaterPosisjon() for å beregne posisjonen
    \n vinkelMellom() for å beregne vinkelen mellom legemet og et annet
    \n tegn() for å tegne legemet
    """
    
    def __init__(self,x_posisjon,y_posisjon,masse,radius,farge,bilde):
        self.x_posisjon = x_posisjon
        self.y_posisjon = y_posisjon
        self.masse = masse
        self.radius = radius
        self.x_fart = 0
        self.y_fart = 0
        self.farge = farge
        self.collision = False
        self.bilde = pygame.transform.scale(bilde, (3*self.radius,3*self.radius))

    # Metode for å finne summen av kreftene
    def tyngdekraft(self, annet_legeme):
        # x og y posisjonen til det andre legemet
        x2_posisjon, y2_posisjon = annet_legeme.x_posisjon, annet_legeme.y_posisjon
        
        # Beregner avstanden mellom legemene
        x_avstand = x2_posisjon - self.x_posisjon
        y_avstand = y2_posisjon - self.y_posisjon
        avstand = math.sqrt(x_avstand**2 + y_avstand**2)

        # Beregner tyngdekraften som virker mellom legemene ved bruk av formel 1
        tyngdekraft = (GRAVITASJONS_KONSTANT * self.masse * annet_legeme.masse) / avstand**2
        theta = math.atan2(y_avstand,x_avstand)

        # Formel 1 og 2
        x_tyngdekraft = math.cos(theta) * tyngdekraft
        y_tyngdekraft = math.sin(theta) * tyngdekraft

        return x_tyngdekraft,y_tyngdekraft
    
    # Metode for å oppdatere posisjon i forhold til andre legemer i rommet
    def oppdaterPosisjon(self,legemer):

        if not self.collision:
            x_totalKrefter = y_totalKrefter = 0

            # Kjører for hvert legeme i lista
            for legeme in legemer:
                if self is legeme:
                    continue
                x_krefter, y_krefter = self.tyngdekraft(legeme)
                x_totalKrefter += x_krefter
                y_totalKrefter += y_krefter

                
            # Bruker at farten er F/m * t
            self.x_fart += x_totalKrefter / self.masse * TIDS_STEG
            self.y_fart += y_totalKrefter / self.masse * TIDS_STEG

            # Endrer posisjonen utifra det
            self.x_posisjon += self.x_fart * TIDS_STEG
            self.y_posisjon += self.y_fart * TIDS_STEG

    # Metode for å finne vinkelen mellom to legemer
    def vinkelMellom(self, annet_legeme):
        # Beregner prikkproduktet
        prikkProduktet = self.x_posisjon * annet_legeme.x_posisjon + self.y_posisjon * annet_legeme.y_posisjon
        
        # Beregner størrelsen på vektoren
        størrelse = math.sqrt(self.x_posisjon**2 + self.y_posisjon**2) * math.sqrt(annet_legeme.x_posisjon**2 + annet_legeme.y_posisjon**2)
        
        # Finner vinkelen
        vinkel = math.degrees(math.acos(prikkProduktet / størrelse))
        return vinkel
    
    # Metode for å tegne legemet
    def tegn(self,vindu):
        # Bestemmer posisjonen til legemet i vinduet
        x = self.x_posisjon * SKALA + SKJERM_BREDDE / 2 - self.radius
        y = self.y_posisjon * SKALA + SKJERM_HØYDE / 2 - self.radius
        
        # Tegner legemet på bilde
        vindu.blit(self.bilde, (x, y))


##### Klassen romfartøy
 
Som nevnt tidligere, trengs to "impulsive motorforbrenninger" for å akselerere og sende romfartøyet ut og inn i bane. For å gjøre det enklere for oss selv, lager vi en klasse som arver metodene fra klassen "Legeme". Videre må vi utlede en formel for å finne farten som trengs for å sende fartøyet inn og ut av ellipsebane:



##### Farten for å sende ut av / inn i ellipsebane

Bruker bevaring av energi for å finne ... 
Setter h=0 ved … 

$E=E_{k}+E_{p}$


$\frac{1}{2}mv_{0}^2=\frac{1}{2}mv^2+(-\frac{γMm}{r})$

Setter inn uttrykket for $v$ inn i ${v_{0}}$
I en ellipse er uttrykket for farten:
 
$v_{e}=\sqrt{\frac{Mγ}{a}}$ 

$\frac{γMm}{2a} = \frac{1}{2}mv^2+(-\frac{γMm}{r})$
  
$a= \frac{r_1+r_2}{2}$


For jorda:
$v_{j}= \sqrt{2γM_{s}\left(\frac{1}{r_{1}} - \frac{1}{r_{1} + r_{2}} \right)}$

For mars:
$v_{s} = \sqrt{γM_{s}\left(\frac{1}{r_{2}} - \frac{1}{r_{1} + r_{2}} \right)}$

Dette bruker vi til å definere de to metodene førsteBoost() og sisteBoost() som justerer farten

In [100]:
class Romfartøy(Legeme):

    def __init__(self, x_posisjon, y_posisjon, masse, radius, farge, bilde):
        super().__init__(x_posisjon, y_posisjon, masse, radius, farge, bilde)

    def førsteBoost(self, radius1, radius2, startsfart):
        # Finner farten som trengs for å sende ut i bane (Vj formelen)
        nødvendigFart = math.sqrt(2 * GRAVITASJONS_KONSTANT * SOL_MASSE * (1/radius1 - 1/(radius1+radius2)))
        
        # Finner faktoren som trengs for å multiplisere x og y komponentene ved å bruke den nåværende farten
        fartsFaktor = nødvendigFart / startsfart

        # Multipliserer farten til romskipet med denne faktoren
        self.x_fart = self.x_fart * fartsFaktor
        self.y_fart = self.y_fart * fartsFaktor
        
    def sisteBoost(self,radius1,radius2,sluttFart):
        # Finner farten som trengs for å sende inn i bane (Vm formelen)
        nødvendigFart = math.sqrt(2 * GRAVITASJONS_KONSTANT * SOL_MASSE * (1/radius2 - 1/(radius1+radius2)))

        # Finner faktoren som trengs for å multiplisere x og y komponentene ved å bruke den nåværende farten
        fartsFaktor = sluttFart / nødvendigFart

        # Multipliserer farten til romskipet med denne faktoren
        self.x_fart = self.x_fart * fartsFaktor
        self.y_fart = self.y_fart * fartsFaktor

##### Omløpshastighet
I tillegg trenger vi å kunne finne omløpshastigheten til jorda og mars for å vite startsfarten og sluttfarten til raketten

$∑F_{j}=G$

$\frac{M_{j}v_{j}^2}{r_{1}}=\frac{γM_{j}M_{s}}{{r_{1}}^2}↔v_{j}=\sqrt{\frac{γM_{s}}{r_{1}}}$



Finner omløpshastighet til jorda rundt sola:

$v_{j}=\sqrt{\frac{γM_{s}}{r_{1}}}=\sqrt{\frac{1,9891\times{10}^{30}\times6.67\times{10}^{-11}}{150\times{10}^9}}=29750m/s$


Finner omløpshastigheten til mars rundt sola: 

$v_{m}=\sqrt{\frac{γM_{s}}{r_{2}}}=\sqrt{\frac{1,9891\times{10}^{30}\times6.67\times{10}^{-11}}{150\times{10}^9\times{1.52}}}=24123m/s$





In [101]:
def omløpshastighet(masse,avstand):
    # Kalkulerer omløpshastigheten
    return math.sqrt((GRAVITASJONS_KONSTANT * masse)/avstand) 

##### Finner når romfartøyen må ut av bane for å nå mars

Keplers tredje lov:

Vinkelgreia:




YANG HJELP!!!

In [102]:
def keplersTredjeLov(radius1,radius2):
    a = (radius1+radius2)/2

    #Gjør om til astronomisk enhet:
    aTilAstronomiskEnhet = a / astronomisk_enhet
    periode = math.sqrt(aTilAstronomiskEnhet**3)
    return periode * 12

def finnUtskytningsVinkel(tid,periode):
    vinkel = (-1*(360*(tid/12)/(periode/12)-180))
    return vinkel


marsTid = keplersTredjeLov(AVSTANDJORD,AVSTANDMARS)/2 #Tiden det tar å dra til mars
marsPeriode = keplersTredjeLov(AVSTANDMARS,AVSTANDMARS) #Perioden til mars
vinkel = finnUtskytningsVinkel(marsTid,marsPeriode)
    

## Setter opp baner og legemer

In [103]:
OmløpshastighetJorda = omløpshastighet(SOL_MASSE,AVSTANDJORD)
OmløpshastighetMars = omløpshastighet(SOL_MASSE,AVSTANDMARS)

# Load sun and mars images
solBilde = pygame.image.load("sola.png")  # Replace "sun_image.png" with the actual filename of your sun image
marsBilde = pygame.image.load("mars.png")  # Replace "mars_image.png" with the actual filename of your mars image
romfartøyBilde = pygame.image.load("romfartøy.png")
jordBilde = pygame.image.load("earth.png")


# Lager planet objektene
# Her ble det gitt uproposjonale radiuser, men det er kun for å modellere, det påvirker ikke utregningen
sola = Legeme(0,0,SOL_MASSE,20, "yellow",solBilde)
mars = Legeme(AVSTANDMARS, 0, MARS_MASSE, 8, "red",marsBilde)
romfartøy = Romfartøy(0,AVSTANDJORD+6371*10**3, 1, 4, "green",romfartøyBilde)
jorda = Legeme(0,AVSTANDJORD,0.0001,10,"blue",jordBilde)

# Snur fortegnet på mars sin startsfart for en rotasjon mot klokken
mars.y_fart = -1 * OmløpshastighetMars

# Setter startfarten på romfartøyen
romfartøy.x_fart = OmløpshastighetJorda
jorda.x_fart = OmløpshastighetJorda


# Load star image
rommet = pygame.image.load("rommet.jpg")  # Replace "star_image.png" with the actual filename of your star image

# Resize star image to match screen size
rommet = pygame.transform.scale(rommet, (SKJERM_BREDDE, SKJERM_HØYDE))

# Draw starry background

legemer = [sola,romfartøy,mars,jorda]

## Initialiserer programmet

In [104]:
pygame.init()
vindu = pygame.display.set_mode((SKJERM_BREDDE,SKJERM_HØYDE))
pygame.display.set_caption("Hohmann-bane")


def main():

    # Nødvendig for pygame-vinduet
    font = pygame.font.Font(None,50)
    clock = pygame.time.Clock()
    teller = 0

    # En feilmargin for når den siste "boosten" skal bli brukt
    feilMargin = (0.5*astronomisk_enhet) / 200
    feilMarginVinkel = 0.01
    # Teller for å sette spor etter romfartøyen for å demonstrere en halv ellipse og en liste for den
    sporTeller = 0
    koordinatListe=[]

    # Boolske variabler som skal brukes
    tegnSpor = False
    førsteBoost = True
    sisteBoost = True

    while True:
        # Nødvendig for pygame-vinduet
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        vindu.blit(rommet, (0, 0))
       

        if mars.vinkelMellom(romfartøy) >= vinkel-feilMarginVinkel and mars.vinkelMellom(romfartøy) <= vinkel+feilMarginVinkel and førsteBoost:
            teller = 0
            romfartøy.førsteBoost(AVSTANDJORD,AVSTANDMARS,math.sqrt(romfartøy.x_fart**2+romfartøy.y_fart**2))
            førsteBoost = False
            tegnSpor = True
            
        # Bestemmer om romfartøyen har nådd målet sitt ved bruk av failmarginen
        if math.sqrt(romfartøy.x_posisjon** 2 + romfartøy.y_posisjon**2) >= (AVSTANDMARS - feilMargin) and math.sqrt(romfartøy.x_posisjon** 2 + romfartøy.y_posisjon**2) <= (AVSTANDMARS + feilMargin) and sisteBoost:
            romfartøy.sisteBoost(AVSTANDJORD,AVSTANDMARS,OmløpshastighetMars)
            sisteBoost = False
        
        # Tegner spor etter romfartøyen:
        if tegnSpor:
            if sporTeller % 75 == 0 and sisteBoost:
                koordinatListe.append([romfartøy.x_posisjon * SKALA, romfartøy.y_posisjon * SKALA])
                sporTeller = 0
            
            sporTeller+=1

            for koordinat in koordinatListe:
                pygame.draw.circle(vindu,"white", (int(koordinat[0] + SKJERM_BREDDE/2), int(koordinat[1] + SKJERM_HØYDE/2)), 2)

        # Tegner banene til jorda og mars
        pygame.draw.circle(vindu, "blue", ( 0+SKJERM_BREDDE/2, 0+SKJERM_HØYDE/2), AVSTANDJORD * SKALA, 1)
        pygame.draw.circle(vindu, "red", ( 0+SKJERM_BREDDE/2, 0+SKJERM_HØYDE/2), AVSTANDMARS * SKALA, 1)

        # Sola skal stå stille
        sola.x_fart, sola.y_fart = 0, 0 

        # Tegner legemene og oppdaterer deres posisjoner:       
        for legeme in legemer:
            legeme.tegn(vindu)
            legeme.oppdaterPosisjon(legemer)

        
        romfartøy.tegn(vindu)
        romfartøyFart = math.sqrt(romfartøy.x_fart**2 + romfartøy.y_fart ** 2)

        # Skriver farten og tiden på vinduet:
        vindu.blit(font.render("Farten til raketten: " + str(int(romfartøyFart)) + " m/s",True,"White"),(10,10))

        if førsteBoost == False and sisteBoost==True:
            teller += TIDS_STEG
        
        vindu.blit(font.render("Tid: " + str(int(teller/(60*60*24))) + " dager",True,"White"),(10,70))
    
        pygame.display.update()

        clock.tick(60)
          
        
main()


SystemExit: 